# A Simple GAN Example in PyTorch
---

## Author : Amir Atapour-Abarghouei, amir.atapour-abarghouei@durham.ac.uk

This notebook will provide an example that shows the implementation of a simple Generative Adversarial Network (GAN).

Copyright (c) 2024 Amir Atapour-Abarghouei, UK.

License : GNU General Public License Version 3

We are going to implement a very simple GAN. Let's start by importing what we need:

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import os
import shutil

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device is {device}!")

We should now line up the dataset we are going to use. We will be working with our own dataset:

**[Radiant X-Ray: A Small Dataset of Processed Chest X-Ray Images](https://github.com/atapour/DGNN-Demos/tree/main/R-X-Ray)**

First, let's download the dataset, which is publicly available on GitHub:

In [ ]:
!wget -q -O R-X-Ray.zip https://github.com/atapour/DGNN-Demos/blob/main/R-X-Ray/R-X-Ray.zip?raw=true
!unzip -q R-X-Ray.zip
!rm R-X-Ray.zip
print('done!')

Now we are ready to process our data. We are going to convert our data to 32x32 images to make the work easier and more efficient just for demonstration purposes.

Since our dataset does not have classes and is only there to train generative models, we are also going to add a fake class directory to make the implementation easier.

In [ ]:
import os
import shutil
import torchvision
import torch

# path to the root directory containing images
root_dir = 'R-X-Ray'

# check if there are already directories in root_dir
subdirectories = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

# if no class directories are found, create one and move the files
if not subdirectories:
    class_dir = os.path.join(root_dir, 'fakeclass')
    os.makedirs(class_dir, exist_ok=True)

    # move all images from the root directory to the new subdirectory
    for file_name in os.listdir(root_dir):
        file_path = os.path.join(root_dir, file_name)
        if os.path.isfile(file_path):
            shutil.move(file_path, os.path.join(class_dir, file_name))

    print(f"Created directory {class_dir} and moved images into it.")
else:
    print(f"Subdirectories already found: {subdirectories}")

# define the transformations
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize([32, 32]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
])

# create the dataset and dataloader using ImageFolder
dataset = torchvision.datasets.ImageFolder(root_dir, transform=transform)

# load the dataset
train_loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True, num_workers=2)

print(f"There are {len(dataset)} images in the training set!")

Our dataset is tiny and is not really suited for any real applications, but it will demonstrate the process. Let's look at a few of our images:

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_loader.dataset[i][0].clamp(0,1).permute(0,2,1).contiguous().permute(2,1,0), cmap=plt.cm.binary)

We can now create our models, we need a Generator and a Discriminator:

In [ ]:
import torch
import torch.nn as nn

# a few parameters:
n_channels = 3
img_width = 32

# define the generator
class Generator(nn.Module):
    def __init__(self, latent_size=100):
        super(Generator, self).__init__()
        self.init_size = img_width // 4
        self.l1 = nn.Sequential(nn.Linear(latent_size, 128 * self.init_size * self.init_size))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, n_channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.l1(x)
        x = x.view(x.size(0), 128, self.init_size, self.init_size)
        x = self.conv_blocks(x)
        return x

# define the discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(n_channels, 64, 3, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 3, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 3, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 3, 2, 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 3, 2, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.model(x)
        return x.view(-1, 1).squeeze(1)

# instantiate the models
G = Generator().to(device)
D = Discriminator().to(device)

# print the number of parameters
print(f'Generator has {sum(p.numel() for p in G.parameters())} parameters.')
print(f'Discriminator has {sum(p.numel() for p in D.parameters())} parameters.')

# initialise the optimisers
optimiser_G = torch.optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimiser_D = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
print('Optimisers have been created!')

# define the loss function
criterion = nn.BCELoss()
epoch = 0
print('Loss function is Binary Cross Entropy!')


Let's start the main training loop now:

In [ ]:
# training loop
while (epoch<20000):

    # arrays for metrics
    logs = {}
    gen_loss_arr = np.zeros(0)
    dis_loss_arr = np.zeros(0)

    # iterate over the train dateset
    for i, batch in enumerate(train_loader):

        x, t = batch
        x, t = x.to(device), t.to(device)

        # train discriminator
        g = G(torch.randn(x.size(0), 100).to(device))
        l_r = criterion(D(x).mean(), torch.ones(1)[0].to(device)) # real -> 1
        l_f = criterion(D(g.detach()).mean(), torch.zeros(1)[0].to(device)) #  fake -> 0
        loss_d = (l_r + l_f)/2.0
        optimiser_D.zero_grad()
        loss_d.backward()
        optimiser_D.step()

        # train generator
        g = G(torch.randn(x.size(0), 100).to(device))
        loss_g = criterion(D(g).mean(), torch.ones(1)[0].to(device)) # fake -> 1
        optimiser_G.zero_grad()
        loss_g.backward()
        optimiser_G.step()

        gen_loss_arr = np.append(gen_loss_arr, loss_g.item())
        dis_loss_arr = np.append(dis_loss_arr, loss_d.item())

    # plot some examples
    G.eval()
    g = G(torch.randn(x.size(0), 100).to(device))
    print('loss D: {:.3f}, loss G: {:.3f}'.format(gen_loss_arr.mean(), dis_loss_arr.mean()))
    plt.grid(False)
    plt.imshow(torchvision.utils.make_grid(g).cpu().data.clamp(0,1).permute(0,2,1).contiguous().permute(2,1,0), cmap=plt.cm.binary)
    plt.show()
    plt.pause(0.0001)
    G.train()

    epoch = epoch+1